# 第5章: 大規模言語モデル

この章では、大規模言語モデル (LLM; Large Language Model) の利用し、様々なタスクに取り組む。大規模言語モデルをプログラムからAPI経由で呼び出すことを想定しており、そのAPIの利用で費用が発生する可能性があることに留意せよ。

## 40. Zero-Shot推論

以下の問題の解答を作成せよ。ただし、解答生成はzero-shot推論とせよ。

```
9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。
イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。
ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 1 問3

In [1]:
!uv pip install openai
!uv pip install python-dotenv

Using Python 3.11.12 environment at: /home/ryuichi/.venv
Resolved 16 packages in 537ms                                        
Prepared 7 packages in 312ms                                             
Installed 7 packages in 19ms                                
 + annotated-types==0.7.0
 + distro==1.9.0
 + jiter==0.9.0
 + openai==1.77.0
 + pydantic==2.11.4
 + pydantic-core==2.33.2
 + typing-inspection==0.4.0
Using Python 3.11.12 environment at: /home/ryuichi/.venv
Resolved 1 package in 178ms                                          
Prepared 1 package in 37ms                                               
Installed 1 package in 2ms0                                 
 + python-dotenv==1.1.0


In [2]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# .envファイルをロードして環境変数を読み込む
load_dotenv()

# 環境変数から値を取得
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# # 環境変数が取得できているか確認
# print(f"Azure Endpoint: {azure_endpoint}")
# print(f"API Key: {api_key}")
# print(f"API Version: {api_version}")

# 必須の環境変数が欠けている場合エラーをスロー
if not azure_endpoint or not api_key or not api_version:
    raise ValueError("必須の環境変数の値が取得できていません。環境変数を確認してください。")

# Azure OpenAI Clientの初期化
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

In [3]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import textwrap

def fetch_responses(payload, system_message="あなたは優秀な日本の歴史の専門家です。"):
    """
    Azure OpenAI APIにリクエストを送り、生成されたレスポンスを返却します。
    """
    try:
        # Temperatureのバリデーション
        if not (0.0 <= payload["temperature"] <= 1.0):
            raise ValueError("temperatureの値は0.0から1.0の間で指定してください。")

        # API呼び出し: Chat Completions
        response = client.chat.completions.create(
            model=payload["model"],
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": payload["prompt"]}
            ],
            temperature=payload["temperature"]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"リクエストに失敗しました。エラーメッセージ: {str(e)}"


# ペイロード設定
payload = {
    "model": "gpt-4o",  # Azure OpenAIでデプロイしたモデルのデプロイメント名を指定
    "prompt": textwrap.dedent("""
        9世紀に活躍した人物に関係するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。
        ア: 藤原時平は、策謀を用いて菅原道真を政界から追放しました。
        イ: 嵯峨天皇は、藤原冬嗣らを蔵人頭に任命しました。
        ウ: 藤原良房は、承和の変後、藤原氏の中での北家の優位を確立しました。
    """).strip(),
    "temperature": 0.7
}

# レスポンス取得
result = fetch_responses(payload)
print(result)

年代の古い順に並べると、正しい順序は以下の通りです：

**イ → ウ → ア**

### 順序の説明：
1. **イ: 嵯峨天皇は、藤原冬嗣らを蔵人頭に任命しました。**
   - 嵯峨天皇が藤原冬嗣を蔵人頭に任命したのは812年のことです。この任命により、藤原氏北家が政界での影響力を増し始めました。

2. **ウ: 藤原良房は、承和の変後、藤原氏の中での北家の優位を確立しました。**
   - 承和の変は842年に発生しました。この政変により、藤原良房が北家の立場を強化し、日本の政治における藤原氏の優位を確立しました。

3. **ア: 藤原時平は、策謀を用いて菅原道真を政界から追放しました。**
   - 藤原時平は、901年に菅原道真を太宰府に左遷する事件（いわゆる「昌泰の変」を引き起こしました）。これにより菅原道真が政界を去ることになりました。

以上を踏まえると、年代の順序は「イ → ウ → ア」となります。


## 41. Few-Shot推論

以下の問題と解答を与え、問題40で示した質問の解答をfew-shot推論（この場合は4-shot推論）で生成せよ。

```
日本の近代化に関連するできごとについて述べた次のア～ウを年代の古い順に正しく並べよ。

ア　府知事・県令からなる地方官会議が設置された。
イ　廃藩置県が実施され，中央から府知事・県令が派遣される体制になった。
ウ　すべての藩主が，天皇に領地と領民を返還した。

解答: ウ→イ→ア
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史A 1 問8


```
江戸幕府の北方での対外的な緊張について述べた次の文ア～ウを年代の古い順に正しく並べよ。

ア　レザノフが長崎に来航したが，幕府が冷淡な対応をしたため，ロシア船が樺太や択捉島を攻撃した。
イ　ゴローウニンが国後島に上陸し，幕府の役人に捕らえられ抑留された。
ウ　ラクスマンが根室に来航し，漂流民を届けるとともに通商を求めた。

解答: ウ→ア→イ
```

出典: [令和5年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00010.htm) [日本史AB 問題](https://www.mext.go.jp/content/20240523-mxt_syogai02-mext_000031286_03nihonshi.pdf) 日本史B 3 問3

```
中居屋重兵衛の生涯の期間におこったできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　アヘン戦争がおこり，清がイギリスに敗北した。
イ　異国船打払令が出され，外国船を撃退することが命じられた。
ウ　桜田門外の変がおこり，大老の井伊直弼が暗殺された。

解答: イ→ア→ウ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 1 問1


```
加藤高明が外務大臣として提言を行ってから、内閣総理大臣となり演説を行うまでの時期のできごとについて述べた次のア～ウを，年代の古い順に正しく並べよ。

ア　朝鮮半島において，独立を求める大衆運動である三・一独立運動が展開された。
イ　関東大震災後の混乱のなかで，朝鮮人や中国人に対する殺傷事件がおきた。
ウ　日本政府が，袁世凱政府に対して二十一カ条の要求を突き付けた。

解答: ウ→ア→イ
```

出典: [令和4年度第1回高等学校卒業程度認定試験問題](https://www.mext.go.jp/a_menu/koutou/shiken/kakomon/1411255_00007.htm) [日本史 問題](https://www.mext.go.jp/content/20240513-mxt_syogai02-mext_00002452_03nihonshi.pdf) 日本史A 2 問4


## 42. 多肢選択問題の正解率

[JMMLU](https://github.com/nlp-waseda/JMMLU) のいずれかの科目を大規模言語モデルに解答させ、その正解率を求めよ。

## 43. 応答のバイアス

問題42において、実験設定を変化させると正解率が変化するかどうかを調べよ。実験設定の例としては、大規模言語モデルの温度パラメータ、プロンプト、多肢選択肢の順番、多肢選択肢の記号などが考えられる。

正解の選択肢を全てDに入れ替えて解答させる例。

## 44. 対話

以下の問いかけに対する応答を生成せよ。

> つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。

参考: [東急線・みなとみらい線路線案内](https://www.tokyu.co.jp/railway/station/map.html)

## 45. マルチターン対話

先ほどの応答に続けて、以下の追加の問いかけに対する応答を生成せよ。

> さらに、つばめちゃんが自由が丘駅で乗り換えたとき、先ほどとは反対方向の急行電車に間違って乗車してしまった場合を考えます。目的地の駅に向かうため、自由が丘の次の急行停車駅で降車した後、反対方向の各駅停車に乗車した場合、何駅先の駅で降りれば良いでしょうか？

## 46. 川柳の生成

適当なお題を設定し、川柳の案を10個作成せよ。

## 47. LLMによる評価

大規模言語モデルを評価者（ジャッジ）として、問題46の川柳の面白さを10段階で評価せよ。

## 48. LLMによる評価の頑健性

問題47で行ったLLMによるテキストの評価に関して、その頑健さ（脆弱さ）を調査せよ。最も単純な方法は、同じ評価を何回か繰り返した時のスコアの分散を調べることであろう。また、川柳の末尾に特定のメッセージを追加することで、評価スコアを恣意的に操作することも可能であろう。

## 49. トークン化

以下の文章（夏目漱石の『吾輩は猫である』の冒頭部分）のトークン数を計測せよ。

>　吾輩は猫である。名前はまだ無い。
>
>　どこで生れたかとんと見当がつかぬ。何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。吾輩はここで始めて人間というものを見た。しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。この書生というのは時々我々を捕えて煮て食うという話である。しかしその当時は何という考もなかったから別段恐しいとも思わなかった。ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。掌の上で少し落ちついて書生の顔を見たのがいわゆる人間というものの見始であろう。この時妙なものだと思った感じが今でも残っている。第一毛をもって装飾されべきはずの顔がつるつるしてまるで薬缶だ。その後猫にもだいぶ逢ったがこんな片輪には一度も出会わした事がない。のみならず顔の真中があまりに突起している。そうしてその穴の中から時々ぷうぷうと煙を吹く。どうも咽せぽくて実に弱った。これが人間の飲む煙草というものである事はようやくこの頃知った。
